In [1]:

import numpy as np
import matplotlib.pyplot as plt
import os
import h5py

In [10]:
# main_path= "R:\Cervical_Project\Data\Healthy"
# main_path= "R:\Cervical_Project\Data\COLPOs"
main_path= "R:\Cervical_Project\Data\Leeps"
os.chdir(main_path)
files = os.listdir()

In [11]:
results_folders=[]
files = os.listdir()
for file in files:
    new_path = os.path.join(main_path, file)
    print(new_path)
    if os.path.isdir(new_path) and 'results_final' in os.listdir(new_path):
        paired_path = os.path.join(new_path, 'results_final')
        results_folders.append(paired_path)

R:\Cervical_Project\Data\Leeps\20190724_#21_LEEP
R:\Cervical_Project\Data\Leeps\20190816_LEEP
R:\Cervical_Project\Data\Leeps\20190920_#24_LEEP
R:\Cervical_Project\Data\Leeps\20200728_LEEP
R:\Cervical_Project\Data\Leeps\20200901_LEEP
R:\Cervical_Project\Data\Leeps\20200902_LEEP
R:\Cervical_Project\Data\Leeps\20200904_LEEP
R:\Cervical_Project\Data\Leeps\20200909_LEEP
R:\Cervical_Project\Data\Leeps\20210126_LEEP
R:\Cervical_Project\Data\Leeps\20210330_LEEP
R:\Cervical_Project\Data\Leeps\20210519_LEEP
R:\Cervical_Project\Data\Leeps\20210604_LEEP
R:\Cervical_Project\Data\Leeps\20210617_LEEP
R:\Cervical_Project\Data\Leeps\20210622_LEEP
R:\Cervical_Project\Data\Leeps\20210810_LEEP
R:\Cervical_Project\Data\Leeps\20210910_LEEP
R:\Cervical_Project\Data\Leeps\20210929_LEEP
R:\Cervical_Project\Data\Leeps\20211022_LEEP
R:\Cervical_Project\Data\Leeps\20211108_LEEP
R:\Cervical_Project\Data\Leeps\20211116_LEEP
R:\Cervical_Project\Data\Leeps\20211126_LEEP
R:\Cervical_Project\Data\Leeps\20211217_LEEP
R:

In [12]:
results_folders

['R:\\Cervical_Project\\Data\\Leeps\\20190724_#21_LEEP\\results_final',
 'R:\\Cervical_Project\\Data\\Leeps\\20190816_LEEP\\results_final',
 'R:\\Cervical_Project\\Data\\Leeps\\20190920_#24_LEEP\\results_final',
 'R:\\Cervical_Project\\Data\\Leeps\\20200728_LEEP\\results_final',
 'R:\\Cervical_Project\\Data\\Leeps\\20200901_LEEP\\results_final',
 'R:\\Cervical_Project\\Data\\Leeps\\20200902_LEEP\\results_final',
 'R:\\Cervical_Project\\Data\\Leeps\\20200904_LEEP\\results_final',
 'R:\\Cervical_Project\\Data\\Leeps\\20200909_LEEP\\results_final',
 'R:\\Cervical_Project\\Data\\Leeps\\20210126_LEEP\\results_final',
 'R:\\Cervical_Project\\Data\\Leeps\\20210330_LEEP\\results_final',
 'R:\\Cervical_Project\\Data\\Leeps\\20210519_LEEP\\results_final',
 'R:\\Cervical_Project\\Data\\Leeps\\20210604_LEEP\\results_final',
 'R:\\Cervical_Project\\Data\\Leeps\\20210617_LEEP\\results_final',
 'R:\\Cervical_Project\\Data\\Leeps\\20210622_LEEP\\results_final',
 'R:\\Cervical_Project\\Data\\Leeps\\202

In [13]:
def normalize(image, p_min=2, p_max=99.9, dtype='float32'):
    '''
    Normalizes the image intensity so that the `p_min`-th and the `p_max`-th
    percentiles are converted to 0 and 1 respectively.
    References
    ----------
    Content-Aware Image Restoration: Pushing the Limits of Fluorescence
    Microscopy
    https://doi.org/10.1038/s41592-018-0216-7
    '''
    low, high = np.percentile(image, (p_min, p_max))
    return (image - low) / (high - low + 1e-6)

def patch_image(image):
    '''Image should be 512x512'''
    return image.reshape(2, 256, 2, 256).swapaxes(1, 2).reshape(4, 256, 256)

def stitch_image(image):
    '''Image should be 4x256x256'''
    return np.reshape(image,[2, -1, 256, 256]).swapaxes(1,2).reshape(512, 512)

def create_patches(input,start,end):
    start = int(start)
    end = int(end)
    indices = list(range(start,end+1))
    total_patches = []
    for i in indices:
        image = input[i]
        image = np.transpose(image)
        image = normalize(image,2,99.9)
        patches = patch_image(image)
        total_patches.append(patches)
    #total_patches = np.swapaxes(total_patches,0,1)\
    total_patches = np.reshape(total_patches,(-1,256,256))
    return list(total_patches)   

In [14]:
size=512
X_N = []
X_F = []
Y_N = []
Y_F = []
start_list = []
end_list = []
key_names = []
ROI_counter = 0
stack_count = 0
for result_folder in results_folders:
    roi_mats = []
    os.chdir(result_folder)
    experiment_files = os.listdir()
    for experiment_file in experiment_files:
        if experiment_file.endswith('FINAL.mat'):
            roi_mats.append(experiment_file)
    for roi in roi_mats:
        roi_name = roi[0:-4]
        key_names.append(roi_name)
        f= h5py.File(roi)
        try: 
            start_stack = f['stack_begin'][0]-1
            end_stack = f['stack_end'][0]-1
            FAD= f['FAD_frames_512_shifted']
            NADH= f['NADH_frames_512_shifted']
            stack_num = end_stack - start_stack +1
            assert stack_num > 0
            low_SNR_NADH = create_patches(NADH[3],start_stack,end_stack)
            high_SNR_NADH = create_patches(np.mean(NADH, axis=0),start_stack,end_stack)
            low_SNR_FAD = create_patches(FAD[3],start_stack,end_stack)
            high_SNR_FAD = create_patches(np.mean(FAD, axis=0),start_stack,end_stack)
            X_N.append(low_SNR_NADH)
            Y_N.append(high_SNR_NADH)
            X_F.append(low_SNR_FAD)
            Y_F.append(high_SNR_FAD)
            ROI_counter += ROI_counter
            start_list.append(stack_count*4)
            stack_count += stack_num 
            end_list.append(stack_count*4-1)
            print('Start and end found: ' + roi_name + ": images=" + str(stack_num))
        except:
            print('Data was not processed: ' + roi_name)
print('done')

Data was not processed: 20190724_#21_LEEP_ROI1_Series001_Series003_FINAL
Data was not processed: 20190724_#21_LEEP_ROI2_Series006_Series008_FINAL
Data was not processed: 20190724_#21_LEEP_ROI3_Series011_Series013_FINAL
Data was not processed: 20190724_#21_LEEP_ROI4_Series016_Series018_FINAL
Data was not processed: 20190816_LEEP_ROI1_Series001_Series003_FINAL
Data was not processed: 20190816_LEEP_ROI2_Series004_Series006_FINAL
Data was not processed: 20190920_#24_LEEP_ROI1_FLIM 002_Series001_FLIM 003_Series003_FINAL
Data was not processed: 20190920_#24_LEEP_ROI2_FLIM 004_Series006_FLIM 005_Series008_FINAL
Data was not processed: 20190920_#24_LEEP_ROI3_FLIM 006_Series011_FLIM 007_Series013_FINAL
Data was not processed: 20190920_#24_LEEP_ROI4_FLIM 011_Series016_FLIM 012_Series018_FINAL
Start and end found: 20200728_LEEP_ROI1_Series005_Series006_FINAL: images=[24.]
Start and end found: 20200728_LEEP_ROI2_Series003_Series004_FINAL: images=[21.]
Data was not processed: 20200728_LEEP_ROI3_Ser

In [15]:
# TODO (nvora01): Fix the way start list saves the initial index of 0
start_list[0]=np.array([int(start_list[0])])
stack_begin = [item for sublist in start_list for item in sublist]
stack_end = [item for sublist in end_list for item in sublist]

In [16]:
X_n = [item for sublist in X_N for item in sublist]
Y_n = [item for sublist in Y_N for item in sublist]
X_f = [item for sublist in X_F for item in sublist]
Y_f = [item for sublist in Y_F for item in sublist]

In [17]:
np.savez('R:\\Cervical_Project\\Data\\NV_907_NADH_Leep', X=X_n, Y=Y_n, SB=stack_begin, SE=stack_end)
np.savez('R:\\Cervical_Project\\Data\\NV_907_FAD_Leep', X=X_f, Y=Y_f, SB=stack_begin, SE=stack_end)

In [ ]:
plt.figure(figsize = [5,5])
plt.imshow(Y_n[0])
plt.figure(figsize = [5,5])
plt.imshow(Y_n[1])
plt.figure(figsize = [5,5])
plt.imshow(Y_n[2])
plt.figure(figsize = [5,5])
plt.imshow(Y_n[3])

In [ ]:
from tensorflow.image import ssim as ssim
x = np.expand_dims(X_n[2],2)
y = np.expand_dims(Y_n[2],2)
print(ssim(x,y,1,k2=0.05))

In [ ]:
n_images=len(X_n)
print(n_images)
n_val   = int(round(n_images * 0.15))
n_train = n_images - n_val
X_t, Y_t = X_n[-n_val:],  Y_n[-n_val:]

In [ ]:
n=133

In [ ]:
n=0
plt.figure(figsize = [10,10])
g= np.reshape(X_n[4*n:4*n+4],[2, -1, 256, 256]).swapaxes(1,2).reshape(512, 512)
#patches= image.reshape(2, 256, -1, 256).swapaxes(1, 2).reshape(-1, 256, 256)
plt.imshow(g,cmap='Greys_r')
g2=np.reshape(Y_n[4*n:4*n+4],[2, -1, 256, 256]).swapaxes(1,2).reshape(512, 512)
plt.figure(figsize = [10,10])
plt.imshow(g2,cmap='Greys_r')

g3=g2.reshape(2, 256, 2, 256).swapaxes(1, 2).reshape(4, 256, 256)
g4=np.reshape(g3[4*n:4*n+4],[2, -1, 256, 256]).swapaxes(1,2).reshape(512, 512)
plt.figure(figsize = [10,10])
plt.imshow(g4,cmap='Greys_r')
# plt.figure(figsize = [5,5])
# plt.imshow(Y_n[0])
# plt.figure(figsize = [5,5])
# plt.imshow(Y_n[1])
# plt.figure(figsize = [5,5])
# plt.imshow(Y_n[2])

In [ ]:
X_val=X_t[4*n+2:4*n+6]
Y_val=Y_t[4*n+2:4*n+6]
print(np.shape(X_val))
print(np.shape(Y_val))

In [ ]:
def normalize(image, p_min=2, p_max=99.9, dtype='float32'):
    '''
    Normalizes the image intensity so that the `p_min`-th and the `p_max`-th
    percentiles are converted to 0 and 1 respectively.
    References
    ----------
    Content-Aware Image Restoration: Pushing the Limits of Fluorescence
    Microscopy
    https://doi.org/10.1038/s41592-018-0216-7
    '''
    low, high = np.percentile(image, (p_min, p_max))
    return (image - low) / (high - low + 1e-6)

In [ ]:
# for p in range(len(X_val)):
#     X_val[p] = normalize(X_val[p],2,99.9)
#     Y_val[p] = normalize(Y_val[p],2,99.9)
g= np.reshape(X_val,[2, -1, 256, 256]).swapaxes(1,2).reshape(512, 512)
g2=np.reshape(Y_val,[2, -1, 256, 256]).swapaxes(1,2).reshape(512, 512)
X_val = normalize(g,2,99.9)
Y_val = normalize(g2,2,99.9)
plt.figure(figsize = [10,10])
g= X_val
plt.imshow(g,cmap='Greys_r')
g2=Y_val
plt.figure(figsize = [10,10])
plt.imshow(g2,cmap='Greys_r')

In [ ]:
plt.figure(figsize = [10,10])
g= np.reshape(X_val,[2, -1, 256, 256]).swapaxes(1,2).reshape(512, 512)
plt.imshow(g,cmap='Greys_r')
g2=np.reshape(Y_val,[2, -1, 256, 256]).swapaxes(1,2).reshape(512, 512)
plt.figure(figsize = [10,10])
plt.imshow(g2,cmap='Greys_r')

In [ ]:
plt.figure(figsize = [5,5])
plt.imshow(Y_val[0],cmap='Greys_r')
plt.figure(figsize = [5,5])
plt.imshow(Y_val[1],cmap='Greys_r')
plt.figure(figsize = [5,5])
plt.imshow(Y_val[2],cmap='Greys_r')
plt.figure(figsize = [5,5])
plt.imshow(Y_val[3],cmap='Greys_r')

In [ ]:
data_path = 'R:\Cervical_Project\Data'
os.chdir(data_path)
np.savez("NV_707_NADH_healthy", X=X_n, Y=Y_n)
np.savez("NV_707_FAD_healthy", X=X_f, Y=Y_f)

In [ ]:
data_path = 'R:\Cervical_Project\Data'
os.chdir(data_path)
f = np.load('HealthyČ_NADH_healthy.npz')
X, Y = f['X'], f['Y']

In [ ]:
plt.imshow(X[3][1])

In [ ]:
np.max(X_n[4][1])


In [ ]:
np.min(X_n[4][1])

In [ ]:
a = (np.linspace(10,95,num=8,dtype="int"))
print(a)